In [ ]:
# Run this with the `integration` kernel, not `cxcr4`


# Make global reference using immune cells


library(Seurat)
library(tidyverse)
library(patchwork)

outdir <- '/home/ubuntu/data/cxcr4-pdac/seurat/'
message_ <- function(m) {
    message(paste(Sys.time(), ': ', m))
}

obj.list <- list()

for (i in seq(18, 40)) {
    if (i == 25) next();
    if (!(i %in% c(18, 20, 21, 28, 30, 32, 35, 37, 38))) next(); # just the samples with unambiguous T cell clusters based on initial predictions

    sample <- paste0('GM', i)
    # message_(paste('Reading sample', sample))
    rds <- file.path(outdir, paste0(sample, '/', sample, '_cb.rds'))
    obj.list[[i]] <- readRDS(rds)
    message_(paste(sample, 'has', dim(obj.list[[i]]@meta.data)[1], 'cells'))
}
obj.list <- obj.list[lengths(obj.list) != 0]  # Reset the index bc we skipped to 18



message_('Normalizing and Finding Variable Features for each sample')
obj.list <- lapply(X = obj.list, FUN = function(x) {
    DefaultAssay(x) <- 'RNA'
    x <- NormalizeData(x, assay='RNA')
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000, assay='RNA')
})
message_('Selecting integration features')
features <- SelectIntegrationFeatures(object.list = obj.list, assay=rep('RNA', length(obj.list)))
message_('Finding integration anchors')
# Credit to Mori for the `smallest_sample` bit. If n_cells < k.score, integration will fail
smallest_sample = min(min(sapply(X = obj.list, FUN = ncol)), 101)-1
anchors <- FindIntegrationAnchors(object.list = obj.list, anchor.features = features, 
                                  # reduction='rpca',  # Mori has beef with RPCA so default is CCA
                                  dims = 1:smallest_sample,  # critical parameter
                                  k.score = smallest_sample, # critical parameter
                                  k.anchor = 5,
                                  k.filter = 200 # Change to NA if necessary
                                  )

message_('Actually integrating')
combined <- IntegrateData(anchorset = anchors, normalization.method='LogNormalize',
                          k.weight = smallest_sample)

message_('Post-integration scaling, reduction, and clustering')
DefaultAssay(combined) <- "integrated"
combined <- ScaleData(combined, verbose = FALSE)
combined <- RunPCA(combined, npcs = 50, verbose = FALSE)
combined <- RunUMAP(combined, reduction = "pca", dims = 1:35)
combined <- FindNeighbors(combined, reduction = "pca", dims = 1:35)
combined <- FindClusters(combined, resolution = 0.5)

saveRDS(combined, file.path(outdir, 'integrated_data.RDS'))  # NOT all samples, just those with unambiguous T cell clusters
# We integrate all samples in a later script

# In retrospect, creating this object wasn't *strictly* necessary since
# we ended up subsetting out the immune cells based solely on SingleR predictions.
# However, it was useful to get confirmation that all the predicted T cells
# did indeed cluster together

# Took about 3-4h total on r5.4xlarge for 22 samples


In [ ]:
library(Seurat)
library(tidyverse)
library(patchwork)

outdir <- '/home/ubuntu/data/cxcr4-pdac/seurat/'
message_ <- function(m) {
    message(paste(Sys.time(), ': ', m))
}
combined <- readRDS(file.path(outdir, 'integrated_data.RDS'))
# reference <- readRDS(file.path(outdir, 'reference.RDS'))
reference <- subset(combined, celltype_bped_main%in%c('CD4+ T-cells', 'CD8+ T-cells', 'NK cells'))
ref_cells <- rownames(reference@meta.data)

# New block
combined <- ScaleData(combined, verbose = FALSE)
combined <- RunPCA(combined, npcs = 30, verbose = FALSE)
combined <- RunUMAP(combined, reduction = "pca", dims = 1:30)
combined <- FindNeighbors(combined, reduction = "pca", dims = 1:30)
combined <- FindClusters(combined, resolution = 0.5)


reference <- subset(combined, celltype_bped_main%in%c('CD4+ T-cells', 'CD8+ T-cells', 'NK cells'))
ref_cells <- rownames(reference@meta.data)
combined@meta.data[rownames(combined@meta.data) %in% ref_cells, 'Reference'] <- TRUE
combined@meta.data$Reference[is.na(combined@meta.data$Reference)] <- 0

# Keep only the most high-quality clusters for our global reference
# This means removing cells from the reference if they are predicted as T but
# make up less than 25% of the cluster
good_clust <- combined@meta.data %>%
    group_by(seurat_clusters) %>%
    summarize(freq = sum(Reference)/n()) %>%
    filter(freq > 0.25) %>%
    .$seurat_clusters

reference <- subset(reference, seurat_clusters %in% good_clust)
ref_cells <- rownames(reference@meta.data)
saveRDS(reference, file.path(outdir, 'reference.RDS'))

p1 <- DimPlot(combined, reduction = "umap", group.by = "celltype_bped_main", label = TRUE,
               repel = TRUE) + theme(legend.position = "none")
p2 <- DimPlot(combined, reduction = "umap", group.by = "seurat_clusters", label = TRUE,
                repel = TRUE) + theme(legend.position = "none")
p3 <- DimPlot(combined, reduction = "umap", group.by = "orig.ident", label = FALSE) + theme(legend.position = "none")
p4 <- DimPlot(object = combined, cells.highlight = ref_cells, cols.highlight = "red", 
        cols = "gray", order = TRUE) + 
        ggtitle(paste0('Reference (n = ', length(ref_cells), ')')) + 
        theme(legend.position = "none")
w <- 4
h <- w * 5
options(repr.plot.width=w, repr.plot.height=h)
print(p1 / p2 / p3 / p4)
ggsave(file.path(outdir, 'reference_subset.png'), height=h, width=w)



In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
DimPlot(combined, reduction = "umap", group.by='orig.ident')

# Downstream Numbat processing

Run these using the `numbat` env

In [ ]:
# Use the `numbat` env for this part

library(numbat)
library(dplyr)
library(Seurat)
library(ggplot2)
library(stringr)
library(patchwork)
library(gifski)

'%notin%' <- Negate('%in%')
message_ <- function(m) {
    message(paste(Sys.time(), ': ', m))
}

################################################################################

# Read-in numbat output
produce_seurat <- function(pat) {
    projectdir <- file.path('/home/ubuntu/data/cxcr4-pdac')
    seu_file <- file.path(projectdir, 'seurat', pat, paste0(pat, '_cb.rds'))
    allele_pileup <- file.path(projectdir, 'numbat', pat, paste0(pat, '_allele_counts.tsv.gz')) # From pileup
    outdir <- file.path(projectdir, 'numbat', pat, 'numbat_downstream')  # From numbat R script
    gif_dir <- file.path(outdir, 'create_gif')

    nb = Numbat$new(out_dir = outdir)
    seu <- readRDS(seu_file)

    # Single-cell CNV calls
    cnv_calls<- nb$joint_post %>% select(cell, CHROM, seg, cnv_state, p_cnv, p_cnv_x, p_cnv_y)

    # Clone info
    clones<-dim(table(nb$clone_post$clone_opt))
    clone_info<-nb$clone_post
    seu$cell<-colnames(seu)
    seu@meta.data<-left_join(seu@meta.data,clone_info,by='cell')
    rownames(seu@meta.data)<-colnames(seu)
    return(list('seurat'=seu, 'gif_dir'=gif_dir))
}

filter_numbat_threshold <- function(seu, outdir,
                                    thresholds=seq(0,1,0.01), 
                                    min_prevalence = 10, 
                                    create_gif = FALSE, delay=0.3,
                                    return_seurat_object = FALSE) {

    if (return_seurat_object) {
        stopifnot('To return seurat object, `thresholds` must be a scalar value' = length(thresholds) == 1)
        stopifnot('To return Seurat object, you `create_gif` must be FALSE' = !create_gif)
    }

    samplename <- seu@meta.data$orig.ident %>% unique()
    stopifnot('Seurat object must only have a single `orig.ident`' = length(samplename) == 1)

    # First, pre-compute prevalence graph for p3
    prevalence <- list()
    ct = 1

    message_(paste('Computing thresholds for', samplename))
    for (thresh in seq(0,1,0.01)) {  # This remains hard-coded
        seu$selected <- seu$p_cnv_y > thresh
        # Cleanup
        prev_df <- seu@meta.data %>% 
                            group_by(seurat_clusters) %>% 
                            summarize(prevalence = 100 * sum(selected)/n()) %>%
                            filter(prevalence > min_prevalence) %>%  # 10% prevalence threshold
                            mutate(threshold = thresh)
        prevalence[[ct]] <- prev_df; ct <- ct + 1
        allowed_clusters <- prev_df$seurat_clusters
        seu$selected_filtered <- seu$selected & (seu$seurat_clusters %in% allowed_clusters)
    }
    prevalence <- dplyr::bind_rows(prevalence)


    message_(paste('Creating images for', samplename))
    pic_ct <- 0
    for (thresh in thresholds) {
        seu$selected <- seu$p_cnv_y > thresh
        # Cleanup
        prev_df <- seu@meta.data %>% 
                            group_by(seurat_clusters) %>% 
                            summarize(prevalence = 100 * sum(selected)/n()) %>%
                            filter(prevalence > min_prevalence) %>%
                            mutate(threshold=thresh)
        allowed_clusters <- prev_df$seurat_clusters
        seu$selected_filtered <- seu$selected & (seu$seurat_clusters %in% allowed_clusters)

        suppressMessages(suppressWarnings({
            p1 <- DimPlot(seu, group.by='selected', order = T) +
            scale_color_manual(values=c('FALSE'='grey', 'TRUE'='red')) +
            ggtitle(paste0(samplename, 
                          ' Tumor vs normal probability (allele)\nThreshold = ', 
                          thresh))

            p2 <- DimPlot(seu, group.by='selected_filtered', order = T) +
                scale_color_manual(values=c('FALSE'='grey', 'TRUE'='red')) +
                ggtitle(paste0(samplename, 
                               ' Tumor vs normal probability (allele)\nThreshold = ', 
                               thresh, '; FILTERED'))

            p3 <- ggplot(prevalence, aes(threshold, prevalence, color=seurat_clusters)) + 
                    geom_line() + 
                    geom_vline(xintercept=thresh, color='red', linetype='dashed') +
                    geom_hline(yintercept=min_prevalence, color='black', linetype='dashed') +
                    xlim(0,1) + ylim(0,100) + 
                    scale_x_continuous(expand = c(0, 0), limits = c(0, NA)) + 
                    scale_y_continuous(expand = c(0, 0), limits = c(0, NA)) +
                    theme_classic() + theme(legend.position = "none") +
                    ylab('% of malignant cells in cluster') +
                    ggtitle(paste0(samplename, 
                                   ' Malignant prevalence per cluster (FILTERED), threshold = ', 
                                   thresh))
        }))

        # Produce plot files
        if (!dir.exists(outdir)) {
            dir.create(outdir, recursive = T)
        }
        outfile <- file.path(outdir, paste0(samplename, '_numbat_thresh_', 
                                            str_pad(thresh*100, 2, pad='0'),
                                            '_min_prev_', min_prevalence,
                                            '.png'))
        ggsave(outfile, ((p1 + p2) / p3),
              width = 12, height = 10, dpi = 300, 
              units = "in", device='png') %>% suppressMessages() %>% suppressWarnings()
        
        if (pic_ct %% 10 == 0) {
            message_(paste0('Produced plot for ', outfile))
        }


        # Will only run if `thresholds` is a single value and we're not creating a gif
        if (return_seurat_object) {
            message_(paste0('Returning Seurat object for ', 
                             samplename, ' threshold ', thresh, 
                             ' and min prevalence ', min_prevalence,'%.'))
            return(seu)
        }
        pic_ct <- pic_ct + 1
    }
    

    if (create_gif) {
        gif_path <- file.path(outdir, paste0(samplename, '_numbat_thresholds.gif'))
        png_files <- file.path(outdir, paste0(samplename, '_numbat_thresh_', 
                                             str_pad(thresholds*100, 2, pad='0'),
                                            '_min_prev_', min_prevalence,
                                            '.png'))
        message_(paste('Creating GIF for sample', samplename))
        gifski::gifski(png_files, gif_path, delay=delay)
        message_(paste('Created GIF at', gif_path))
        file.remove(png_files)
    }
}


In [ ]:
# Now, I just create gifs to help select clusters within each sample that 
# have the highest prevalence of malignant cells as predicted by Numbat, 
# while filtering out clusters with low signal

In [ ]:
seu <- produce_seurat('GM18')
# Run this cell multiple times with various values of 
# `thresholds` and `min_prevalence` until you get good separation
# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

# # Finally, after you've tweaked the results and decided on an optimum
# # This will add the new boolean columns `selected` and `selected_filtered`
seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.43,
                                min_prevalence=30,
                                create_gif=F,
                                return_seurat_object=T)
# # Now you're free to saveRDS and do downstream analysis on malignant cells

In [ ]:
seu <- produce_seurat('GM19')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.10,
                                min_prevalence=75,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM20')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.12,
                                min_prevalence=30,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM21')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.30,
                                min_prevalence=75,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM23')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.15, 
                                min_prevalence=75,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM26')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.15, 
                                min_prevalence=85,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM27')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.05, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM28')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.15, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM30')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.20, 
                                min_prevalence=85, # ???
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM31')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.05, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM32')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.1, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM33')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.45, 
                                min_prevalence=60,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM37')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=30,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.15, 
                                min_prevalence=80,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM38')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=10,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.05, 
                                min_prevalence=50,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM39')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=20,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.15, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T)

In [ ]:
seu <- produce_seurat('GM40')

# filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
#                         thresholds=seq(0,1,0.01),
#                         min_prevalence=10,
#                         create_gif=T, 
#                         delay=0.3  # Only matters if `create_gif` is TRUE
#                         )

seu <- filter_numbat_threshold(seu[['seurat']], outdir=seu[['gif_dir']],
                                thresholds=0.20, 
                                min_prevalence=90,
                                create_gif=F,
                                return_seurat_object=T) 